In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
#sys.path.insert(0, '/Users/ammar.tareen/Desktop/Research_projects/mavenn2/')
sys.path.insert(0, '/Users/tareen/Desktop/Research_Projects/2022_mavenn2_github/mavenn')

import mavenn
import logomaker
import seaborn as sns
import re

%matplotlib inline

In [2]:
# Load example data
data_df = mavenn.load_example_dataset('sortseq')

# Separate test from data_df
ix_test = data_df['set']=='test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'test N: {len(test_df):,}')

# Remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'training + validation N: {len(data_df):,}')
data_df.head()

test N: 9,935
training + validation N: 40,583


,set,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,ct_9,x
0,training,0,1,0,0,0,0,0,0,0,0,AAAAAAAGTGAGTTAGCCAACTAATTAGGCACCGTACGCTTTATAG...
1,training,0,0,0,0,0,0,0,0,0,1,AAAAAATGAGAGTTAGTTCACTCATTCGGCACCACAGGCTTTACAA...
2,training,0,0,0,0,0,0,0,0,0,1,AAAAAATGGGTGTTAGCTCTATCATTAGGCACCCCCGGCTTTACAC...
3,training,0,1,0,0,0,0,0,0,0,0,AAAAAATGTCAGTTAGCTGACTCATTAGGCACCCCTGGCTTTACGT...
4,validation,0,0,0,0,0,0,1,0,0,0,AAAAAATGTGAGAAAGCTCACTCCTTTGGCACCGCAGGCTTTACAC...


In [3]:
# set positional argumnets for gpmap function
L = len(data_df['x'][0])
alphabet=['A','C','G','T']

y_cols = data_df.columns[1:-1]
Y = len(y_cols)
print(f'L={L}, Y={Y}')

# get test set data.
x_test = test_df['x'].values

L=75, Y=10


In [4]:
# instantiate gpmap object
gpmap = mavenn.gpmap.AdditiveGPMapLayer(L, alphabet)


2022-05-27 14:38:15.077519: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Create measurement process object(s)

In [5]:
# I think this shouldn't require any positional arguments.
# mp_GE = mavenn.measurement_process_layers.GlobalEpistasisLayer(K=50,
#                                                                eta=1e-5,
#                                                                monotonic=True)
mp_mpa = mavenn.measurement_process_layers.DiscreteMonotonicMP(info_for_layers_dict={'H_y_norm':0},
                                                                      Y=10,
                                                                      K=50,
                                                                      eta=1e-5,)


In [6]:
model = mavenn.Model2(gpmap=gpmap,
                      mp_list=[mp_mpa])

masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_6:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_7:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_8:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_9:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_10:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_11:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_12:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("discrete_monotonic_mp/Reshape_13:0", shape=(None, 10), dtype=float

In [7]:
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Sequence_labels_input (InputLa  [(None, 310)]       0           []                               
 yer)                                                                                             
                                                                                                  
 Sequence_only (Lambda)         (None, 300)          0           ['Sequence_labels_input[0][0]']  
                                                                                                  
 additive_gp_map_layer (Additiv  (None, 1)           301         ['Sequence_only[0][0]']          
 eGPMapLayer)                                                                                     
                                                                                              

In [8]:
# #*np.ones(shape=y_list[0].shape[0])

# y_cts_df = pd.DataFrame(np.random.rand(y_list[0].shape[0]))
# y_cts_df.head()

In [9]:
# y_dict = {}

# y_list=[data_df[y_cols].values]
# #y_dict[output_layer_index] = current_y_norm
# y_discrete_df = pd.DataFrame(y_list[0])
# y_discrete_df.head()

In [10]:
# np.hstack([model.x_ohe,np.hstack([y_cts_df.values,y_discrete_df.values])])

In [11]:

#np.hstack([y_cts_df.values])

In [12]:
# Set training data
model.set_data(x=data_df['x'].values,
               y_list=[data_df[y_cols].values],
               validation_flags=(data_df['set']=='validation'),
               shuffle=True)


N = 40,583 observations set as training data.
Using 24.8% for validation.
Data shuffled.
Time to set data: 0.943 sec.


In [13]:

# Fit model to data
model.fit(learning_rate=.001,
          epochs=1000,
          batch_size=200,
          try_tqdm = False,
          early_stopping=True,
          early_stopping_patience=10,
          linear_initialization=False)

Epoch 1/1000
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_6:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_7:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_8:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_9:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_10:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_11:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discrete_monotonic_mp/Reshape_12:0", shape=(None, 10), dtype=float32)
masked tensor shape (None, None)
masked tensor  Tensor("model/discr

InvalidArgumentError:  Input to reshape is a tensor with 928 values, but the requested shape requires a multiple of 10
	 [[node model/discrete_monotonic_mp/Reshape_14
 (defined at /Users/tareen/Desktop/Research_Projects/2022_mavenn2_github/mavenn/mavenn/src/layers/measurement_process_layers.py:1381)
]] [Op:__inference_train_function_3698]

Errors may have originated from an input operation.
Input Source operations connected to node model/discrete_monotonic_mp/Reshape_14:
In[0] model/discrete_monotonic_mp/boolean_mask_8/GatherV2 (defined at /Users/tareen/Desktop/Research_Projects/2022_mavenn2_github/mavenn/mavenn/src/layers/measurement_process_layers.py:1378)	
In[1] model/discrete_monotonic_mp/Reshape_14/shape:

Operation defined at: (most recent call last)
>>>   File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/Library/Frameworks/Python.framework/Versions

In [ ]:
# Show training history
print('On test data:')
x_test = test_df['x'].values
y_test = test_df[y_cols].values


I_var_hist = model.history['I_var']
val_I_var_hist = model.history['val_I_var']

fig, ax = plt.subplots(1,1,figsize=[4,4])
ax.plot(I_var_hist, label='I_var_train')
ax.plot(val_I_var_hist, label='I_var_val')
ax.legend()
ax.set_xlabel('epochs')
ax.set_ylabel('bits')
ax.set_title('training hisotry')

In [ ]:
# Get G-P map parameters in matrix form
theta = np.squeeze(gpmap.weights[1].numpy())

# Create grid in phi space
phi_lim = [-5, 3]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Create array of allowable y values
Y = mp_mpa.Y    # Y = number of bins
y_lim = [-.5, Y-.5]
y_all = range(Y)

# Compute matrix of p(y|phi) values
#measurement_process = mp_mpa.p_of_y_given_phi(y_all, phi_grid)

# Create figure with two panels
fig, axs = plt.subplots(1,1,figsize=[10,4])

# Left panel: draw logo using logomaker
ax = axs
logo = logomaker.Logo(pd.DataFrame(theta,columns=model.alphabet), ax=ax,center_values=True)
ax.set_ylabel(r'parameter value ($\theta_{l:c}$)')
ax.set_xlabel(r'position ($l$)')
ax.set_title('G-P map parameters')

# Right panel: draw measurement process as heatmap
# ax = axs[1]
# im = ax.imshow(measurement_process,
#                cmap='Greens',
#                extent=phi_lim+y_lim,
#                vmin=0,
#                origin='lower',
#                interpolation='nearest',
#                aspect="auto")
# ax.set_yticks(y_all)
# ax.set_ylabel('bin number (y)')
# ax.set_xlabel('latent phenotype ($\phi$)')
# ax.set_title('measurement process')
# cb = plt.colorbar(im)
# cb.set_label('probability  $p(y|\phi)$', rotation=-90, va="bottom")

# Fix up plot
fig.tight_layout(w_pad=3)
#fig.savefig('sortseq_g-p_map_mpa_visualization.png')
plt.show()

In [ ]:
# Create grid in phi space
phi_lim = [-5, 3]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Create array of allowable y values
Y = mp_mpa.Y    # Y = number of bins
y_lim = [-.5, Y-.5]
y_all = range(Y)

# Compute matrix of p(y|phi) values
measurement_process = mp_mpa.p_of_y_given_phi(y_all, phi_grid)


# Create figure with two panels
fig, ax = plt.subplots(1,1,figsize=[6,6])

# Right panel: draw measurement process as heatmap
im = ax.imshow(measurement_process,
               cmap='Greens',
               extent=phi_lim+y_lim,
               vmin=0,
               origin='lower',
               interpolation='nearest',
               aspect="auto")
ax.set_yticks(y_all)
ax.set_ylabel('bin number (y)')
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_title('measurement process')
cb = plt.colorbar(im)
cb.set_label('probability  $p(y|\phi)$', rotation=-90, va="bottom")


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K

c = tf.constant([1,2,3,4,5,6,7,8,9,10])
#mask = np.array([True, False, True, False])
#tf.math.segment_sum(c, tf.constant([0,1,1,2,4])).numpy()

In [ ]:
mask = np.full((10,), False, dtype=bool)

In [ ]:
mask

In [ ]:
np.full((1,10), False, dtype=bool)

In [ ]:
np.zeros(shape=(1,10))

In [ ]:
mask = np.full((10,), False, dtype=bool)
for y_prime in range(Y):
    
    mask[0:y_prime]=True
    
    masked_tensor = tf.boolean_mask(c, mask)
    print(K.sum(masked_tensor).numpy())
    #print(K.sum(c[0:y_prime]))
    
#c[0]+c[1]

In [ ]:
#dir(masked_tensor)
masked_tensor

In [ ]:
var = tf.Variable(initial_value=c,shape=c.shape)
var

In [ ]:
var[0].assign(5)

In [ ]:
var